# Prédiction de notes de films Allociné en fonction d'avis


In [10]:
import os
import sys

import numpy as np

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

Nous utiliserons le jeu de données `allocine`

In [11]:
from datasets import load_dataset
dataset = load_dataset("allocine")

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'label'],
        num_rows: 160000
    })
    validation: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
})

In [13]:
train = dataset["train"]
test = dataset["test"]
val = dataset["validation"]

In [14]:
print(train["review"][0])
print(train["label"][0])

Si vous cherchez du cinéma abrutissant à tous les étages,n'ayant aucune peur du cliché en castagnettes et moralement douteux,"From Paris with love" est fait pour vous.Toutes les productions Besson,via sa filière EuropaCorp ont de quoi faire naître la moquerie.Paris y est encore une fois montrée comme une capitale exotique,mais attention si l'on se dirige vers la banlieue,on y trouve tout plein d'intégristes musulmans prêts à faire sauter le caisson d'une ambassadrice américaine.Nauséeux.Alors on se dit qu'on va au moins pouvoir apprécier la déconnade d'un classique buddy-movie avec le jeune agent aux dents longues obligé de faire équipe avec un vieux lou complètement timbré.Mais d'un côté,on a un Jonathan Rhys-meyers fayot au possible,et de l'autre un John Travolta en total délire narcissico-badass,crâne rasé et bouc proéminent à l'appui.Sinon,il n'y a aucun scénario.Seulement,des poursuites débiles sur l'autoroute,Travolta qui étale 10 mecs à l'arme blanche en 8 mouvements(!!)ou laiss

Commençons par nettoyer les avis de tous les mots qui peuvent être "parasites".

Importons `TextualPreparator` et définissons le nettoyage que nous voulons appliquer

In [15]:
from easyTrainer.data.text import TextualPreparator

In [16]:
TP = TextualPreparator(
    to_lower=1,
    drop_stopwords=2,
    drop_digits=3,
    drop_special_characters=4,
    drop_accents=5,
    lemmatize=6,
    drop_words_less_than_N_letters=(7, 4, False),
    drop_big_spaces=8
)

print(TP)

TextualPreparator with transformations: txt_to_lower@1, drop_stopwords@2, drop_digits@3, drop_special_characters@4, drop_accents@5, lemmatize@6, drop_words_less_than_N_letters@7, drop_big_spaces@8


Nous pouvons lancer la préparation. En sortie, nous voulons des données encodées, c'est pourquoi nous utilisons `encoder_name_to_fit="tfidf"`.

In [17]:
train_prep = TP.prepare(train["review"], encoder_name_to_fit="tfidf")

Cela nous retourne nos données nettoyées, encodées, et notre encoder `TfidfVectorizer`. Nous pourrons ensuite l'appliquer sur les données de test et de validation

In [19]:
X_train = train_prep["encoded_data"]
y_train = train["label"]
vectorizer = train_prep["vectorizer"]

Préparons nos données de test et de validation en appliquant notre encodeur entrainé avec `custom_encoder_fit=vectorizer`.

In [20]:
test_prep = TP.prepare(test["review"], custom_encoder_fit=vectorizer)
val_prep = TP.prepare(val["review"], custom_encoder_fit=vectorizer)

X_test = test_prep["encoded_data"]
y_test = test["label"]

X_val = val_prep["encoded_data"]
y_val = val["label"]

Entrainons un modèle

In [21]:
from sklearn.ensemble import RandomForestClassifier

from easyTrainer.models.classifier import SklearnClassifierModel

In [22]:
SCM = SklearnClassifierModel(
    model=RandomForestClassifier(n_estimators=100, random_state=42),
)

In [23]:
SCM.fit(
    X = X_train,
    y = y_train
)

RandomForestClassifier(random_state=42)

In [24]:
SCM.test(
    X= X_test, 
    y= y_test
)

,0,1,accuracy,macro avg,weighted avg
precision,0.893306,0.878597,0.8862,0.885951,0.886251
recall,0.887298,0.885008,0.8862,0.886153,0.886200
f1-score,0.890292,0.881791,0.8862,0.886041,0.886215
support,10408.000000,9592.000000,0.8862,20000.000000,20000.000000


In [30]:
SCM.validation(
    X = X_val, 
    y = y_val,
    fallback_class=0,
    validation_thresholds=np.arange(0.5, 1.0, 0.05)
)

{'thresholds': [np.float64(0.5),
  np.float64(0.55),
  np.float64(0.6000000000000001),
  np.float64(0.6500000000000001),
  np.float64(0.7000000000000002),
  np.float64(0.7500000000000002),
  np.float64(0.8000000000000003),
  np.float64(0.8500000000000003),
  np.float64(0.9000000000000004),
  np.float64(0.9500000000000004)],
 'accuracy': [0.8795,
  0.87845,
  0.85955,
  0.82465,
  0.77975,
  0.72645,
  0.6665,
  0.60295,
  0.54825,
  0.51765],
 'f1_score_weighted': [0.8795117855958988,
  0.8782890596807407,
  0.8583302672034099,
  0.8206827737274119,
  0.7697637619586096,
  0.7044476103485018,
  0.6229784121783493,
  0.5245181759607068,
  0.4250069354388711,
  0.36128290666813917],
 'f1_score_macro': [0.8794805614197514,
  0.8781698879016884,
  0.8580306579433203,
  0.8200990635509908,
  0.768734096777693,
  0.7027347740619513,
  0.6202757017017535,
  0.5204608431081608,
  0.41942797501049645,
  0.35466693922238585],
 'f1_per_classes': {np.int64(0): [np.float64(0.8810111582897205),
   n

In [36]:
text_to_predict = ["Un film MAUVAIS, je ne recommande pas!"]
text_to_predict_prep = TP.prepare(
    text_to_predict,
    custom_encoder_fit=vectorizer
)

SCM.predict(text_to_predict_prep["encoded_data"])

array([0])

In [26]:
# SCM.save(path="yout_path.pkl")